In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **PRE-PROCESSING**

In [ ]:
############################################################# PREPROCESSING

import numpy as np 
import pandas as pd 
import nltk
import string

from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

def remove_stopwords(text) :
  arr_of_words=text.split()
  return_text=''
  for word in arr_of_words :
    if word not in stop :
      return_text+=word
      return_text+=' '
  return return_text

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

def make_lower(text) :
  lower_text=text.lower()
  return lower_text

def remove_numbers(text) :
  alpha_text=''
  arr_of_words=text.split()
  for i in arr_of_words :
    try :
      x=int(i)
    except ValueError :
      alpha_text+=i
      alpha_text+=' '
  return alpha_text

def deal_media(text) :
  arr_of_words=text.split()
  final_text=''
  for i in arr_of_words :
    if 'http://' in i :
      word='aszxdcfvgb'
    else :
      word=i
    final_text+=word
    final_text+=' '
  return final_text

def remove_nan(text) :
  text = text.replace(' nan ', '')
  return text



train_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ML_Project/ML_data/train.csv") 
test_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ML_Project/ML_data/test.csv")

#train_df = pd.read_csv("/content/drive/My Drive/ML_data/train.csv") 
#test_df = pd.read_csv("/content/drive/My Drive/ML_data/test.csv")

#adding location to text
train_df["text"] = train_df["text"] + ' ' + train_df["location"].astype(str) +' '
test_df["text"] = test_df["text"] + ' ' + test_df["location"].astype(str) +' '


train_df["text"] = train_df["text"].apply(remove_nan)
train_df["text"] = train_df["text"].apply(deal_media)
train_df["text"] = train_df["text"].apply(remove_stopwords)
train_df["text"] = train_df["text"].apply(remove_punctuations)
train_df["text"] = train_df["text"].apply(make_lower)
train_df["text"] = train_df["text"].apply(remove_numbers)

test_df["text"] = test_df["text"].apply(remove_nan)
test_df["text"] = test_df["text"].apply(deal_media)
test_df["text"] = test_df["text"].apply(remove_stopwords)
test_df["text"] = test_df["text"].apply(remove_punctuations)
test_df["text"] = test_df["text"].apply(make_lower)
test_df["text"] = test_df["text"].apply(remove_numbers)


x_train=train_df["text"]
y_train=train_df["target"]
x_test=test_df["text"]


print("Preprocessing done")
print("")
print("Locations added to text")
print("Links were removed")
print("Stopwords removed")
print("punctuations removed")
print("Numbers removed")
print("Lowered the text")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Preprocessing done

Locations added to text
Links were removed
Stopwords removed
punctuations removed
Numbers removed
Lowered the text


# **VECTORIZATION**



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

cv = CountVectorizer()
tv = TfidfVectorizer()

x_traincv = cv.fit_transform(x_train)
x_traintv = tv.fit_transform(x_train)

x_testcv = cv.transform(x_test)
x_testtv = tv.transform(x_test)

# **Word2Vec vectorizarion**

In [ ]:
# tried a large corpus, but failed
from tensorflow.keras.utils import get_file

try:
    path = get_file('GoogleNews-vectors-negative300.bin.gz', origin='https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz')
except:
    print('Error downloading')
    raise
    
print(path)
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)

1647050752/1647046227 [==============================] - 103s 0us/step
/root/.keras/datasets/GoogleNews-vectors-negative300.bin.gz


In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')
import gensim
from gensim import corpora, models, models, similarities


train_corpus=x_train
tok_train = [nltk.word_tokenize(sent) for sent in train_corpus]
test_corpus=x_test
tok_test = [nltk.word_tokenize(sent) for sent in test_corpus]


train_model = gensim.models.Word2Vec(tok_train, min_count=1, size=32)
test_model = gensim.models.Word2Vec(tok_test, min_count=1, size=32)

#no_word_list=['ronge', 'sask', 'rockyfire', 'cafire', 'manitou', 'a', 'fvck', 'tampabay', 'goooooooaaaaaal','looooool', 'wayi', 'cooool', 'bbcmtd','aszxdcfvgb', 'africanbaze', 'newsnigeria']

x_trainwv=[]
for sentence in tok_train :
  total=np.zeros((1, 32))
  total=total.tolist()
  total=total[0]
  total=np.array(total)
  len=0
  for word in sentence :
    #if word not in no_word_list:
    len+=1
    #vector_arr=train_model.wv[word].tolist()
    vector_arr=train_model.wv[word].tolist()
    vector_arr=np.array(vector_arr)
    total+= vector_arr
  total=total/len
  total=total.tolist()
  x_trainwv.append(total)
x_trainwv=np.array(x_trainwv)



x_testwv=[]
for sentence in tok_test :
  total=np.zeros((1, 32))
  total=total.tolist()
  total=total[0]
  total=np.array(total)
  len=0
  for word in sentence :
    len+=1
    #vector_arr=test_model.wv[word].tolist()
    vector_arr=test_model.wv[word].tolist()
    vector_arr=np.array(vector_arr)
    total+= vector_arr
  total=total/len
  total=total.tolist()
  x_testwv.append(total)
x_testwv=np.array(x_testwv)




print("Vectorization done for train and test data")
    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Vectorization done for train and test data


# **MULTI LAYER PERCEPTRON**

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
classifier = MLPClassifier()


list_of_parameters_for_hidden_layer_sizes=[(64, 64,), (128, 64,)]
list_of_parameters_for_activation=['logistic', 'relu']
list_of_parameters_for_solver=['sgd', 'adam']
list_of_parameters_for_learning_rate_init=[0.001, 0.002]
list_of_parameters_for_max_iter=[500]


params={'hidden_layer_sizes':list_of_parameters_for_hidden_layer_sizes, 'activation':list_of_parameters_for_activation, 'solver':list_of_parameters_for_solver, 'learning_rate_init':list_of_parameters_for_learning_rate_init, 'max_iter':list_of_parameters_for_max_iter}
gs=GridSearchCV(estimator=classifier, param_grid=params, cv=5)


gs = gs.fit(x_trainwv, y_train)
best_params=gs.best_params_
accuracy=gs.best_score_
print("For Multi-layer perceptron :")
print("Best hyperparameter :", best_params)
print("Accuracy of train data :", accuracy)


For Multi-layer perceptron :
Best hyperparameter : {'activation': 'relu', 'hidden_layer_sizes': (64, 64), 'learning_rate_init': 0.001, 'max_iter': 500, 'solver': 'adam'}
Accuracy of train data : 0.6369388172420607


# **ACCURACY METRICS**

In [4]:

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

clf = MLPClassifier(hidden_layer_sizes=(64, 64,), activation='relu', solver='adam', learning_rate_init=0.001, max_iter=500)

# Count Vectorization
y_pred = cross_val_predict(clf, x_traincv, y_train, cv=5)
accuracy = accuracy_score(y_train, y_pred)
print("Count Vectorization :")
print("Accuracy :", accuracy)
f1 = f1_score(y_train, y_pred)
print("F1 Score :", f1)
conf_mat = confusion_matrix(y_train, y_pred)
print("Confusion Matrix :")
print(conf_mat)
print("")


#TFID Vectorization
y_pred = cross_val_predict(clf, x_traintv, y_train, cv=5)
accuracy = accuracy_score(y_train, y_pred)
print("TFID Vectorization :")
print("Accuracy :", accuracy)
f1 = f1_score(y_train, y_pred)
print("F1 Score :", f1)
conf_mat = confusion_matrix(y_train, y_pred)
print("Confusion Matrix :")
print(conf_mat)
print("")


#Word2Vec vectorization
y_pred = cross_val_predict(clf, x_trainwv, y_train, cv=5)
accuracy = accuracy_score(y_train, y_pred)
print("Word2Vec Vectorization :")
print("Accuracy :", accuracy)
f1 = f1_score(y_train, y_pred)
print("F1 Score :", f1)
conf_mat = confusion_matrix(y_train, y_pred)
print("Confusion Matrix :")
print(conf_mat)


Count Vectorization :
Accuracy : 0.7267713122290818
F1 Score : 0.6434339505212386
Confusion Matrix :
[[2931 1411]
 [1273 1998]]

TFID Vectorization :
Accuracy : 0.7374451595954289
F1 Score : 0.6582035928143714
Confusion Matrix :
[[3093 1249]
 [1364 1907]]

Word2Vec Vectorization :
Accuracy : 0.6891869171154604
F1 Score : 0.5486680873734684
Confusion Matrix :
[[3386  956]
 [1867 1404]]


# **On Test Data**

In [ ]:
import pandas as pd

clf = MLPClassifier(hidden_layer_sizes=(64, 64,), activation='relu', solver='adam', learning_rate_init=0.001, max_iter=500)

#Count Vectorization
clf.fit(x_traincv, y_train)
y_pred=list(clf.predict(x_testcv))
id_arr=list(test_df["id"])
target_arr=y_pred
df = pd.DataFrame(list(zip(id_arr, target_arr)),columns =['id', 'target'])
df.to_csv('Submission_file_mlp_cv.csv', index=False)
!cp Submission_file_mlp_cv.csv "drive/My Drive/"
print("cv file made")
#f1 score is 0.74869

#TFID Vectorization
clf.fit(x_traintv, y_train)
y_pred=list(clf.predict(x_testtv))
id_arr=list(test_df["id"])
target_arr=y_pred
df = pd.DataFrame(list(zip(id_arr, target_arr)),columns =['id', 'target'])
df.to_csv('Submission_file_mlp_tv.csv', index=False)
!cp Submission_file_mlp_tv.csv "drive/My Drive/"
print("tv file made")
#f1 score is 0.74072

#Word2Vec Vectorization
clf.fit(x_trainwv, y_train)
y_pred=list(clf.predict(x_testwv))
id_arr=list(test_df["id"])
target_arr=y_pred
df = pd.DataFrame(list(zip(id_arr, target_arr)),columns =['id', 'target'])
df.to_csv('Submission_file_mlp_wv.csv', index=False)
!cp Submission_file_mlp_wv.csv "drive/My Drive/"
print("wv file made")
#f1 score is 0.58075

cv file made
tv file made
wv file made
